In [24]:
import pandas as pd
import numpy as np
from scipy import stats\

df = pd.read_csv("Practical Example - Confidence Intervals.xlsx - Al Bundy.csv",skiprows=3, header=0)
df.set_index("InvoiceNo", inplace = True)
df = df.drop(columns = ["Unnamed: 0", "SalePrice", "Discount", "UnitPrice", "Size (Europe)", "Size (UK)", "Date"])
df.rename(columns = {"Unnamed: 12" : "Year"}, inplace=True)


test_df = pd.read_csv("Quantico.xlsx - Mutual Funds Performance.csv", skiprows=3, header=0)
test_df = test_df.iloc[:-5]
test_df = test_df.drop(columns = ["Unnamed: 0", "Date"])

def no_symbol(df):
    for col in df.columns:
        df[col] = df[col].str.replace("%","")

def to_float(df):
    for col in df.columns:
        if df[col].dtypes == object:
            df[col] = df[col].astype(float)
    
no_symbol(test_df)
to_float(test_df)

Making data frames to separate year, country and gender

In [25]:
men2016_us = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Male') & (df["Country"] == "United States")]
men2016_cn = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Male') & (df["Country"] == "Canada")]
men2016_uk = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Male') & (df["Country"] == "United Kingdom")]
men2016_ger = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Male') & (df["Country"] == "Germany")]

fem2016_us = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Female') & (df["Country"] == "United States")]
fem2016_cn = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Female') & (df["Country"] == "Canada")]
fem2016_uk = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Female') & (df["Country"] == "United Kingdom")]
fem2016_ger = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Female') & (df["Country"] == "Germany")] 


Making a frequency distribution table

In [26]:
grp_month = men2016_us.groupby("Month")
freq_table = grp_month["Size (US)"].value_counts().unstack().fillna(0)

freq_table

Size (US),6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,10.5,11.0,11.5,12.0,13.0,14.0,15.0
Month,,,,,,,,,,,,,,,,
1,4.0,3.0,0.0,3.0,7.0,12.0,17.0,19.0,17.0,13.0,5.0,4.0,3.0,1.0,2.0,0.0
2,1.0,2.0,0.0,2.0,9.0,12.0,13.0,25.0,26.0,16.0,16.0,3.0,0.0,1.0,6.0,0.0
3,3.0,0.0,1.0,3.0,7.0,8.0,13.0,27.0,26.0,22.0,13.0,6.0,0.0,2.0,3.0,0.0
4,1.0,1.0,0.0,1.0,3.0,8.0,11.0,24.0,19.0,14.0,10.0,3.0,4.0,0.0,3.0,1.0
5,3.0,0.0,6.0,7.0,12.0,15.0,21.0,26.0,16.0,28.0,10.0,3.0,4.0,3.0,5.0,1.0
6,3.0,0.0,4.0,0.0,2.0,9.0,22.0,33.0,31.0,19.0,11.0,5.0,4.0,2.0,3.0,0.0
7,3.0,1.0,4.0,7.0,9.0,17.0,25.0,25.0,25.0,18.0,15.0,6.0,3.0,1.0,2.0,4.0
8,4.0,7.0,2.0,3.0,4.0,17.0,30.0,47.0,24.0,15.0,8.0,4.0,12.0,0.0,1.0,0.0
9,3.0,2.0,3.0,4.0,7.0,6.0,26.0,31.0,23.0,19.0,9.0,5.0,4.0,0.0,0.0,0.0


Calculating the mean and standar error with the frequency table

In [27]:
every_mean = freq_table.mean().round(2)
column_mean = freq_table[6.0].mean()

every_stderr = freq_table.sem().round(2)
column_stderr = freq_table[6.0].sem().round(2)

Function to print confidence intervals

In [41]:
confidence_interval = 0.90

def conf_int_t(df, confidence):
    for col in df.columns:
        col_data = df[col]
        mean = np.mean(col_data)
        std_error = stats.sem(col_data)
        lower_bound, upper_bound = stats.t.interval(confidence, len(col_data) - 1, mean, std_error)
        mean = round(np.mean(col_data), 2)
        lower_bound = round(lower_bound, 2)
        upper_bound = round(upper_bound, 2)
        print(f"{col}: Mean={mean}, confidence={confidence}% Confidence Interval=({lower_bound}, {upper_bound})")

In [42]:
conf_int_t(test_df, confidence_interval)

Quant A: Mean=0.26, confidence=0.9% Confidence Interval=(-3.06, 3.58)
Quant B: Mean=-3.78, confidence=0.9% Confidence Interval=(-6.64, -0.92)
Quant C: Mean=3.21, confidence=0.9% Confidence Interval=(-0.36, 6.79)
